In [96]:
import pandas as pd

In [97]:
# path of all the files that i need
PHI_route = "../data/raw/Average_Daily_Ridership_By_Route - City of Philadelphia.csv"
PHI_mode = "../data/raw/Average_Daily_Ridership_By_Mode - City of Philadelphia.csv"
CHI_mode = "../data/raw/cta-ridership-daily-boarding-totals-20260203-69820a3f9df63091665572.xlsx"
CHI_route = "../data/raw/chicago_ridership_all_routes.csv"

In [98]:
# Chargement des fichiers dans des dataframes
df_phi_route = pd.read_csv(PHI_route)
df_phi_mode = pd.read_csv(PHI_mode)
df_chi_mode = pd.read_excel(CHI_mode, engine="openpyxl")
df_chi_route = pd.read_csv(CHI_route)

C:\Users\mhdme\AppData\Local\Temp\ipykernel_16460\770747477.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_chi_route = pd.read_csv(CHI_route)


# **Nettoyage des Fichiers CHICAGO**

### **FICHIER MODE**

In [99]:
print(df_chi_mode.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9100 entries, 0 to 9099
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   service_date    9100 non-null   datetime64[ns]
 1   day_type        9100 non-null   object        
 2   bus             9100 non-null   int64         
 3   rail_boardings  9100 non-null   int64         
 4   total_rides     9100 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 355.6+ KB
None


In [100]:
# Nettoyage du fichier excel : "Supprimer les lignes < 2019"
df_chi_mode = df_chi_mode[~df_chi_mode['service_date'].dt.year.between(2001, 2018)]
df_chi_mode.info()
df_chi_mode.describe()


<class 'pandas.core.frame.DataFrame'>
Index: 2526 entries, 6574 to 9099
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   service_date    2526 non-null   datetime64[ns]
 1   day_type        2526 non-null   object        
 2   bus             2526 non-null   int64         
 3   rail_boardings  2526 non-null   int64         
 4   total_rides     2526 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 118.4+ KB


,service_date,bus,rail_boardings,total_rides
count,2526,2526.000000,2526.000000,2.526000e+03
mean,2022-06-16 12:00:00,447388.560570,335343.043943,7.827316e+05
min,2019-01-01 00:00:00,80783.000000,23544.000000,1.100470e+05
25%,2020-09-23 06:00:00,307571.500000,203752.000000,5.030232e+05
50%,2022-06-16 12:00:00,441554.500000,322465.000000,7.631590e+05
75%,2024-03-08 18:00:00,566131.500000,405493.000000,9.707490e+05
max,2025-11-30 00:00:00,905477.000000,816086.000000,1.677559e+06
std,NaN,175712.572371,174488.280933,3.456848e+05


Mettre la meme structure du fichier du philadelphie

In [101]:
# Supprimer total_amounts
df_chi_mode = df_chi_mode.drop(columns=['total_rides'])

In [102]:
# Pivoter les colonnes bus et raily_boarding
df_chi_mode = pd.melt(
    df_chi_mode,
    id_vars=['service_date', 'day_type'],
    value_vars=['bus', 'rail_boardings'],
    var_name='Mode',
    value_name='average_monthly_ridership'
)

In [103]:
df_chi_mode.head(15)

,service_date,day_type,Mode,average_monthly_ridership
0,2019-01-01,U,bus,248879
1,2019-01-02,W,bus,591006
2,2019-01-03,W,bus,664442
3,2019-01-04,W,bus,668812
4,2019-01-05,A,bus,444434
5,2019-01-06,U,bus,318623
6,2019-01-07,W,bus,723023
7,2019-01-08,W,bus,784085
8,2019-01-09,W,bus,747949
9,2019-01-10,W,bus,747850


In [104]:
# Extraire les mois et les annees
df_chi_mode['Calendar_Year'] = df_chi_mode['service_date'].dt.year
df_chi_mode['Calendar_Month'] = df_chi_mode['service_date'].dt.month

In [105]:
df_chi_mode = (
    df_chi_mode
    .groupby(['Calendar_Year', 'Calendar_Month', 'Mode'], as_index=False)
    .agg({'average_monthly_ridership': 'mean'})
)
df_chi_mode.head(15)
df_chi_mode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166 entries, 0 to 165
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Calendar_Year              166 non-null    int32  
 1   Calendar_Month             166 non-null    int32  
 2   Mode                       166 non-null    object 
 3   average_monthly_ridership  166 non-null    float64
dtypes: float64(1), int32(2), object(1)
memory usage: 4.0+ KB


In [106]:
# Renommer la colonne avg_moth en majuscule
df_phi_mode.rename(
    columns={ 'average_monthly_ridership':'Average_Monthly_Ridership'},
    inplace=True
)

### **FICHIER ROUTE**

In [107]:
df_chi_route.info()
df_chi_route["date"] = pd.to_datetime(df_chi_route["date"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092438 entries, 0 to 1092437
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   route     1092438 non-null  object
 1   date      1092438 non-null  object
 2   day_type  1092438 non-null  object
 3   rides     1092438 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 33.3+ MB


In [108]:
# Nettoyage du fichier excel : "Supprimer les lignes < 2019"
df_chi_route = df_chi_route[~df_chi_route['date'].dt.year.between(2001, 2018)]
df_chi_route.info()
df_chi_route.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 288924 entries, 3 to 1091544
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   route     288924 non-null  object        
 1   date      288924 non-null  datetime64[ns]
 2   day_type  288924 non-null  object        
 3   rides     288924 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 11.0+ MB


,date,rides
count,288924,288924.000000
mean,2022-06-17 22:01:41.972837376,3879.732553
min,2019-01-01 00:00:00,1.000000
25%,2020-09-24 00:00:00,868.000000
50%,2022-06-20 00:00:00,2562.000000
75%,2024-03-11 00:00:00,5755.000000
max,2025-11-30 00:00:00,27202.000000
std,NaN,3898.287411


Meme structure pour les fichiers routes

In [109]:
# Extraire les mois et les annees
df_chi_route['Calendar_Year'] = df_chi_route['date'].dt.year
df_chi_route['Calendar_Month'] = df_chi_route['date'].dt.month

In [110]:
# Rides daily to Rides monthly
df_chi_route = (
    df_chi_route
    .groupby(['Calendar_Year', 'Calendar_Month', 'route'], as_index=False)
    .agg({'rides': 'mean'})
)
df_chi_route.head(15)
df_chi_route.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10714 entries, 0 to 10713
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Calendar_Year   10714 non-null  int32  
 1   Calendar_Month  10714 non-null  int32  
 2   route           10714 non-null  object 
 3   rides           10714 non-null  float64
dtypes: float64(1), int32(2), object(1)
memory usage: 251.2+ KB


In [111]:
# Renommer la colonne avg_daily en monthly
df_phi_route.rename(
    columns={ 'rides':'Rides'},
    inplace=True
)

# **Nettoyage des fichiers PHILADELPHIE**

### **FICHIER MODE**

In [112]:
df_phi_mode.head(15)

,Calendar_Year,Calendar_Month,Mode,Average_Daily_Ridership,Source,ObjectId
0,2019,1,Bus,459160,APC,1
1,2019,1,CCT,4294,Revenue,2
2,2019,1,Heavy Rail,296709,Revenue,3
3,2019,1,Regional Rail,122856,Revenue,4
4,2019,1,Trackless Trolley,16853,APC,5
5,2019,1,Trolley,82787,APC,6
6,2019,2,Bus,464674,APC,7
7,2019,2,CCT,4652,Revenue,8
8,2019,2,Heavy Rail,315858,Revenue,9
9,2019,2,Regional Rail,121161,Revenue,10


In [113]:
# Filtrer Bus ou tout ce qui contient "Rail"
df_phi_mode = df_phi_mode = df_phi_mode[df_phi_mode['Mode'].eq('Bus') | df_phi_mode['Mode'].str.contains('Rail')]
df_phi_mode.head(15)
df_phi_mode.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246 entries, 0 to 489
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Calendar_Year            246 non-null    int64 
 1   Calendar_Month           246 non-null    int64 
 2   Mode                     246 non-null    object
 3   Average_Daily_Ridership  246 non-null    int64 
 4   Source                   246 non-null    object
 5   ObjectId                 246 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 13.5+ KB


In [114]:
# Rassembler tous les rail en une colonne
df_phi_mode['Mode'] = df_phi_mode['Mode'].replace({
    'Heavy Rail': 'rail_boardings',
    'Regional Rail': 'rail_boardings'
})

In [115]:
df_phi_mode.head(15)

,Calendar_Year,Calendar_Month,Mode,Average_Daily_Ridership,Source,ObjectId
0,2019,1,Bus,459160,APC,1
2,2019,1,rail_boardings,296709,Revenue,3
3,2019,1,rail_boardings,122856,Revenue,4
6,2019,2,Bus,464674,APC,7
8,2019,2,rail_boardings,315858,Revenue,9
9,2019,2,rail_boardings,121161,Revenue,10
12,2019,3,Bus,478487,APC,13
14,2019,3,rail_boardings,308777,Revenue,15
15,2019,3,rail_boardings,118228,Revenue,16
18,2019,4,Bus,473732,APC,19


In [116]:
# Recalculer "average_daily_rideship"
df_phi_mode = (
    df_phi_mode.groupby(
        ['Calendar_Year', 'Calendar_Month', 'Mode'],
        as_index=False
    )['Average_Daily_Ridership']
    .mean()
)

In [117]:
df_phi_mode.head(15)


,Calendar_Year,Calendar_Month,Mode,Average_Daily_Ridership
0,2019,1,Bus,459160.0
1,2019,1,rail_boardings,209782.5
2,2019,2,Bus,464674.0
3,2019,2,rail_boardings,218509.5
4,2019,3,Bus,478487.0
5,2019,3,rail_boardings,213502.5
6,2019,4,Bus,473732.0
7,2019,4,rail_boardings,214768.0
8,2019,5,Bus,480074.0
9,2019,5,rail_boardings,211481.5


In [118]:
# Renommer la colonne avg_daily en monthly
df_phi_mode.rename(
    columns={ 'Average_Daily_Ridership':'Average_Monthly_Ridership'},
    inplace=True
)

### **FICHIER ROUTE**


In [120]:
df_phi_route.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10994 entries, 0 to 10993
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Calendar_Year            10994 non-null  int64 
 1   Calendar_Month           10994 non-null  int64 
 2   Route                    10994 non-null  object
 3   Average_Daily_Ridership  10994 non-null  int64 
 4   Source                   10994 non-null  object
 5   ObjectId                 10994 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 515.5+ KB


In [121]:
# Supprimer les deux colonnes pour avoir la meme structure des fichiers routes
df_phi_route.drop(columns=['Source', 'ObjectId'], inplace=True)

In [122]:
# Renommer la colonne pour garantir la meme structure
df_phi_route.rename(
    columns={'Average_Daily_Ridership':'Rides'},
    inplace=True
)

# **Sauvegarder les dfs_clean en csv :**

In [125]:
import os

os.makedirs("../data/raw/Clean_Data/", exist_ok=True)

df_phi_route.to_csv("../data/raw/Clean_Data/PHI_route.csv", index=False)
df_chi_route.to_csv("../data/raw/Clean_Data/CHI_route.csv", index=False)
df_phi_mode.to_csv("../data/raw/Clean_Data/PHI_mode.csv", index=False)
df_chi_mode.to_csv("../data/raw/Clean_Data/CHI_mode.csv", index=False)